In [1]:
import numpy as np
from sklearn.ensemble import IsolationForest
import asyncio
import platform

In [2]:
# Simulate synthetic behavioral data
def generate_synthetic_data(n_samples=1000, n_features=5, anomaly_ratio=0.05):
    np.random.seed(42)
    normal_data = np.random.normal(0, 1, (int(n_samples * (1 - anomaly_ratio)), n_features))
    anomalous_data = np.random.uniform(-5, 5, (int(n_samples * anomaly_ratio), n_features))
    data = np.vstack([normal_data, anomalous_data])
    labels = np.array([1] * len(normal_data) + [-1] * len(anomalous_data))
    return data, labels

In [3]:
# Behavioral Authentication System
class SecureBBA:
    def __init__(self, contamination=0.05):
        self.model = IsolationForest(contamination=contamination, random_state=42)
        self.threshold = -0.5  # Adjustable threshold for anomaly detection
        self.is_trained = False

    def train(self, data):
        self.model.fit(data)
        self.is_trained = True
        print("Model trained successfully.")

    def detect_anomaly(self, sample):
        if not self.is_trained:
            raise ValueError("Model must be trained before detecting anomalies.")
        score = self.model.score_samples([sample])[0]
        return score, score < self.threshold

    def respond_to_anomaly(self, score, is_anomaly):
        if not is_anomaly:
            return "Normal behavior detected."
        if score < -0.7:
            return "High-risk anomaly detected: Session terminated."
        elif score < -0.5:
            return "Medium-risk anomaly: Restricting high-value transactions."
        else:
            return "Low-risk anomaly: Requesting additional verification."

In [4]:
# Main simulation loop
async def main():
    # Generate synthetic data (typing speed, swipe speed, tap pressure, device tilt, navigation time)
    data, labels = generate_synthetic_data(n_samples=1000, n_features=5, anomaly_ratio=0.05)

    # Initialize and train the model
    bba = SecureBBA(contamination=0.05)
    bba.train(data)

    # Simulate real-time authentication for 10 samples
    test_samples = np.random.normal(0, 1, (10, 5))  # Normal user behavior
    test_samples[8] = np.random.uniform(-5, 5, 5)  # Introduce an anomaly
    test_samples[9] = np.random.uniform(-5, 5, 5)  # Another anomaly

    for i, sample in enumerate(test_samples):
        score, is_anomaly = bba.detect_anomaly(sample)
        response = bba.respond_to_anomaly(score, is_anomaly)
        print(f"Sample {i+1}: Score={score:.3f}, Anomaly={is_anomaly}, Response={response}")
        await asyncio.sleep(0.1)  # Simulate real-time processing


In [7]:
# Run the simulation
# Check if the environment is Emscripten (e.g., Pyodide)
if platform.system() == "Emscripten":
    # Use ensure_future for environments that require explicit task scheduling
    asyncio.ensure_future(main())
else:
    # For standard Python environments like Jupyter,
    # directly await the async function instead of using asyncio.run()
    # which starts a new event loop. Jupyter often manages its own loop.
    # We also wrap it in an async definition to use await.
    async def run_main():
        await main()

    # Call the async wrapper function
    await run_main()

/usr/lib/python3.11/threading.py:254: RuntimeWarning: coroutine 'main' was never awaited
  self._release_save = lock._release_save


Model trained successfully.
Sample 1: Score=-0.400, Anomaly=False, Response=Normal behavior detected.
Sample 2: Score=-0.393, Anomaly=False, Response=Normal behavior detected.
Sample 3: Score=-0.387, Anomaly=False, Response=Normal behavior detected.
Sample 4: Score=-0.396, Anomaly=False, Response=Normal behavior detected.
Sample 5: Score=-0.365, Anomaly=False, Response=Normal behavior detected.
Sample 6: Score=-0.399, Anomaly=False, Response=Normal behavior detected.
Sample 7: Score=-0.480, Anomaly=False, Response=Normal behavior detected.
Sample 8: Score=-0.453, Anomaly=False, Response=Normal behavior detected.
Sample 9: Score=-0.674, Anomaly=True, Response=Medium-risk anomaly: Restricting high-value transactions.
Sample 10: Score=-0.616, Anomaly=True, Response=Medium-risk anomaly: Restricting high-value transactions.
